In [ ]:
import csv
import re
import pickle
import json
import sys
import requests
from pprint import pprint
import os
import datetime
import Methods.classifier as classifier

In [ ]:
with open("sorted_all_dataset.pkl", "rb") as f:
    dataset = pickle.load(f)
    
with open("dataset_dict_all.pkl", "rb") as f:
    data_all = pickle.load(f)

In [ ]:
directory = 'Repos_metrics'

mapping = {}
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):        
        name_splitted = filename.split('_')
        pair_id = int(re.findall(r'\d+', name_splitted[0])[0])

        source = dataset[pair_id]['source']
        dest = dataset[pair_id]['destination']
            
        mapping[pair_id] = {}
        for repo_pair in data_all:
            if data_all[repo_pair]['mainline'] == source:
                if data_all[repo_pair]['variant'] == dest:
                    mapping[pair_id]['repo_id'] = repo_pair
                    mapping[pair_id]['mainline-variant'] = True
            elif data_all[repo_pair]['mainline'] == dest:
                if data_all[repo_pair]['variant'] == source:
                    mapping[pair_id]['repo_id'] = repo_pair
                    
                    mapping[pair_id]['mainline-variant'] = False

In [ ]:
token_list = list()

with open('/mnt/c/Users/User1/Desktop/tokens.txt') as f:
    lines = f.readlines()
    for line in lines[0].split(','):
        token_list.append(line)
        
lenTokens = len(token_list)

In [ ]:
directory = 'Repos_metrics'

time_lag_data = [['pair_id', 'source_variant', 'target_variant', 'mainline-varaint-dir', 'pr_nr', 'pr_title', 'patch_class', 'patched_date', 'latest_date_target', 'lag']]
ct = 0

for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):       
        with open(file_path, 'rb') as f:
            metrics = pickle.load(f)[0]
            
            name_splitted = filename.split('_')
            pair_id = int(re.findall(r'\d+', name_splitted[0])[0])
            
            source = dataset[pair_id]['source']
            dest = dataset[pair_id]['destination']
            try:
                for pr, classes in metrics.items():
                    if classes['verdict'] in [ 'MO', 'SP']:
                        if ct == lenTokens:
                            ct = 0
                        pr_url = 'https://api.github.com/repos/' + source + '/pulls/' + pr
                        pr_data = classifier.apiRequest(pr_url, token_list[ct])
                        ct += 1

                        latest_date = datetime.datetime.strptime(dataset[pair_id]['cut_off_date'], '%Y-%m-%dT%H:%M:%SZ')
                        merged_date = datetime.datetime.strptime(pr_data["merged_at"], '%Y-%m-%dT%H:%M:%SZ')
                        time_lag = latest_date - merged_date
                        time_lag_data.append([mapping[pair_id]['repo_id'], source, dest, mapping[pair_id]['mainline-variant'], pr, pr_data['title'], classes['verdict'], merged_date, latest_date, time_lag])
            except Exception as e:
                print(e)
                print(source, dest)
                print(pr_url)
                print(pr)

In [ ]:
for i in time_lag_data[1:]:
    i[7] = str(i[7])
    i[8] = str(i[8])
    i[9] = str(i[9])
    i.append(str(i[9]))
    i.append(str((i[9].days * 24 *60 * 60)+(i[9].seconds)))

In [ ]:
with open('time lag.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(time_lag_data)